In [ ]:
# A set of functions that can apply different transformations on an existing image to synthesis a new image

from PIL import Image, ImageEnhance, ImageFilter
from skimage import io, data, transform, filters, color, util
import colorsys
import numpy
import math

# Sets plugin for skimage, using PIL to keep read in image formats the same for arrays
io.use_plugin('pil')

In [ ]:
# Change contrast, that eventually can return the negative at high enough values
def change_contrast(img, level):
    factor = (259 * (level + 255)) / (255 * (259 - level))
    def contrast(c):
        value = 128 + factor * (c - 128)
        return max(0, min(255, value))
    return img.point(contrast)

In [ ]:
# Range is real numbers greater than 0 to infinity
def change_contrast_alt(img, level):
    contrast = ImageEnhance.Contrast(img)
    return contrast.enhance(level)

In [ ]:
# Range is real numbers greater than 0 to infinity
def change_brightness(img, level):
    brightness = ImageEnhance.Brightness(img)
    return brightness.enhance(level)

In [ ]:
# Flip image over the vertical axis
def flip_vertical(img):
    return img.transpose(Image.FLIP_LEFT_RIGHT)

In [ ]:
# Flip image over the horizontal axis
def flip_horizontal(img):
    return img.transpose(Image.FLIP_TOP_BOTTOM)

In [ ]:
# Flip image over both axis
def flip_diagonal(img):
    imgcpy = img.transpose(Image.FLIP_TOP_BOTTOM)
    return imgcpy.transpose(Image.FLIP_LEFT_RIGHT)

In [ ]:
# By passing a new_size we have to consider that it may be under, which we technically don't want as we would just be
# creating copies of the image based on the code below
# Need to consider checking for size before this function if we choose to randomize the values, 
# or else we'll end up with alot of doubles
def pad_image(img, new_size):

    old_img = img
    old_size = old_img.size
    
    # Check that all dimensions are greater or equal so it doesn't crop
    if all(i >= j for i, j in zip(new_size, old_size)):
        new_img = Image.new("RGB", new_size)   ## luckily, this is already black!
        new_img.paste(old_img, (int((new_size[0]-old_size[0])/2), int((new_size[1]-old_size[1])/2)))
        return new_img
    else:
        return old_img

In [ ]:
# Skew image using some math
def skew_image(imagepath, angle):
    img = Image.open(imagepath)
    width, height = img.size
    
    # Get the width that is to be added to the image based on the angle of skew
    xshift = math.tan(abs(angle)) * height
    new_width = width + int((xshift))
    
    # Apply transform
    img = img.transform((new_width, height), Image.AFFINE,
            (1, angle, -xshift if angle > 0 else 0, 0, 1, 0), Image.BICUBIC)
    return img

In [ ]:
# Seam carve image
def seam_carve_image(imagepath):
    # Load the image as a matrix
    img = io.imread(imagepath, plugin='pil')
    
    # Energy Map, used to determine which pixels will be removed
    eimg = filters.sobel(color.rgb2gray(img))
    
    # (Width, Height)
    img_Dimensions = img.shape
    
    # Squish width if width >= height, squish height if height > width
    if(img_Dimensions[0] >= img_Dimensions[1]):
        mode = 'vertical'
    else:
        mode = 'horizontal'
    
    # Number of seams to be removed, need to determine best way to randomize
    num_seams = 15
    
    # Number of pixels to keep along the outer edges
    border = 5
    
    return transform.seam_carve(img, eimg, mode, num_seams)

In [ ]:
# Rotate image
def rotate(image_path, output, rotation_angle):
    try:
        im = Image.open(image_path)
        im_rotated = im.rotate(rotation_angle)
        im_rotated.save(output, "JPEG")
    except IOError as e:
        print(e)

In [ ]:
def scale(image_path, output, scaling_factor):
    try:
        im = Image.open(image_path)
        im.thumbnail((original_height*scaling_factor, original_width*scaling_factor), Image.ANTIALIAS)
        im.save(output, "JPEG")
    except IOError as e:
        print(e)

In [ ]:
# Crop image
# TODO: this method still needs to be tweaked so that we dont kill the image (main obj is still visible)
def crop(image_path, output, scaling_factor_x, scaling_factor_y):
    try:
        im = Image.open(image_path)
        im = im.crop((0, 0, int(original_width*scaling_factor_x), int(original_height*scaling_factor_y)))
        im.save(output, "JPEG")
    except Exception as e:
        print(e)

In [ ]:
# Apply white noise to image
def white_noise(image_path, output):
    image = skimage.io.imread(image_path)
    image = skimage.util.img_as_ubyte(util.random_noise(image, mode='s&p', seed=None, clip=True))
    image = skimage.util.img_as_ubyte(util.random_noise(image, mode='gaussian', seed=None, clip=True))
    image = skimage.util.img_as_ubyte(util.random_noise(image, mode='speckle', seed=None, clip=True))
    skimage.io.imsave(output, image)

In [ ]:
def sharpen(image_path, output):
    im = Image.open(image_path)
    im = im.filter(ImageFilter.SHARPEN)
    im.save(output, "JPEG")

In [ ]:
def soften(image_path, output):
    im = Image.open(image_path)
    im = im.filter(ImageFilter.SMOOTH)
    im.save(output, "JPEG")

In [ ]:
def grayscale(image_path, output):
    ''' grayscale
    im = Image.open(image_path) # open colour image
    im = im.convert('1') # convert image to black and white
    im.save(output, "JPEG")
    '''
    
    # black and white
    col = Image.open(image_path)
    gray = col.convert('L')
    bw = gray.point(lambda x: 0 if x<128 else 255, '1')
    bw.save(output)

In [1]:
def hue_change(image_path, output):
    im = Image.open(image_path)
    ld = im.load()
    for y in range(original_height):
        for x in range(original_width):
            r,g,b = ld[x,y]
            h,s,v = colorsys.rgb_to_hsv(r/255., g/255., b/255.)
            h = (h + -90.0/360.0) % 1.0
            s = s**0.65
            r,g,b = colorsys.hsv_to_rgb(h, s, v)
            ld[x,y] = (int(r * 255.9999), int(g * 255.9999), int(b * 255.9999))
    im.save(output, "JPEG")